# 랜덤 포레스트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np 
import pandas as pd # 시리즈, 데이터프레임 포맷 데이터 처리를 위한 라이브러리
import warnings; warnings.filterwarnings(action='ignore') # 경고 메시지 무시
import matplotlib.pyplot as plt # 데이터 시각화 라이브러리
import pickle # 객체 입출력을 위한 라이브러리

from sklearn.model_selection import train_test_split # 훈련 데이터, 테스트 데이터 분리
from sklearn.preprocessing import StandardScaler # 정규화

from pandas.core.common import random_state
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트 분류 알고리즘
from sklearn.tree import DecisionTreeClassifier # 의사결정나무 분류 알고리즘

import matplotlib.pyplot as plt
import seaborn as sns

# 모델 평가를 위한 metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## 데이터 불러오기

In [3]:
# Disease 

df = pd.read_csv('/content/drive/MyDrive/health/data/data_over_sampling(smote).csv', encoding='cp949') # 원본 그대로 오버샘플링(1:3:1) : 스모트
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,116,78,94,16.600000,4,0
1,0,1,100,60,79,22.300000,4,0
2,0,1,100,60,87,21.900000,4,0
3,0,1,111,70,72,20.200000,4,0
4,0,1,120,80,98,20.000000,4,0
...,...,...,...,...,...,...,...,...
1481319,0,27,119,64,94,21.927633,1,1
1481320,0,27,145,70,108,25.343427,1,1
1481321,1,16,123,72,104,20.760203,2,1
1481322,0,27,115,75,143,21.647994,1,1


In [4]:
df.drop(df[df['Dis']==0].index, inplace=True)
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
52,0,1,90,50,86,20.100000,2,1
97,0,1,120,80,90,27.000000,3,1
133,0,1,100,60,112,31.800000,2,1
204,0,1,170,120,86,23.300000,2,1
255,0,1,135,80,104,35.000000,2,1
...,...,...,...,...,...,...,...,...
1481319,0,27,119,64,94,21.927633,1,1
1481320,0,27,145,70,108,25.343427,1,1
1481321,1,16,123,72,104,20.760203,2,1
1481322,0,27,115,75,143,21.647994,1,1


In [5]:
# 1,0 정답 레이블 삭제
df.drop(['Dis'], axis=1, inplace=True)
df

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS
52,0,1,90,50,86,20.100000,2
97,0,1,120,80,90,27.000000,3
133,0,1,100,60,112,31.800000,2
204,0,1,170,120,86,23.300000,2
255,0,1,135,80,104,35.000000,2
...,...,...,...,...,...,...,...
1481319,0,27,119,64,94,21.927633,1
1481320,0,27,145,70,108,25.343427,1
1481321,1,16,123,72,104,20.760203,2
1481322,0,27,115,75,143,21.647994,1


In [6]:
df.notnull().sum()

SEX      740662
BTH_G    740662
SBP      740662
DBP      740662
FBS      740662
BMI      740662
DIS      740662
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 740662 entries, 52 to 1481323
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   SEX     740662 non-null  int64  
 1   BTH_G   740662 non-null  int64  
 2   SBP     740662 non-null  int64  
 3   DBP     740662 non-null  int64  
 4   FBS     740662 non-null  int64  
 5   BMI     740662 non-null  float64
 6   DIS     740662 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 45.2 MB


In [8]:
# 데이터 정규화

# 표준 스케일러(평균 0, 분산 1)
scaler = StandardScaler()

# feature와 target 분리(설명변수 및 목표변수 분리)
x = df.drop(['DIS'], axis=1) # df[df.columns.difference(['target'])]
y = np.array(df['DIS']) # y = df['target']

# 설명변수 데이터 스케일링
x_train = scaler.fit_transform(x) # x_scaled = scaler.fit_transform(x)

In [9]:
# 전체 데이터에서 학습데이터셋과 테스트셋을 나눔
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, shuffle=True) # 기본 0.25

# train과 test데이터의 비율을 6:4로 나누어주고
# 0.4비율이 된 test데이터를 다시 각 0.5 비율로 validation과 test를 나누어 줌
# 최종 결과가 train : test : validation = 6 : 2 : 2 비율

# # train set에서 train과 validation set를 나눔 # x,y,test_size=0.4
# x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5, shuffle=True)

print(x_train.shape)
print(y_train.shape)
# print(x_val.shape)
# print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(592529, 6)
(592529,)
(148133, 6)
(148133,)


In [10]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
forest_predictions = rf.predict(x_test)

accuracy_score(y_test, forest_predictions)
forest_predictions = rf.predict(x_test)

In [11]:
# 데이터 100만 개 사용
print(confusion_matrix(y_test, forest_predictions))

[[25243 10207  1323]
 [ 5986 86916  2508]
 [ 2361  5427  8162]]


In [12]:
rf = RandomForestClassifier(n_estimators=100, random_state=42).fit(x_train,y_train)

print("train score : {}".format(rf.score(x_train, y_train)))
#print("val score : {}".format(rf.score(x_val, y_val)))

#print("test score : {}".format(dTree.score(x_test,y_test)))
predict_y = rf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(x_test)))
print("test score : {}".format(rf.score(x_test, y_test)))

train score : 0.9979984102043951
              precision    recall  f1-score   support

           1       0.75      0.69      0.72     36773
           2       0.85      0.91      0.88     95410
           3       0.68      0.51      0.58     15950

    accuracy                           0.81    148133
   macro avg       0.76      0.70      0.73    148133
weighted avg       0.81      0.81      0.81    148133

test score : 0.8122498025423099


# SMOTE + 질병 1, 2, 3 비율 => 1 : 1 : 1

In [13]:
# Disease 

df2 = pd.read_csv('/content/drive/MyDrive/health/data/data(over_dis_vs_nor_smote).csv', encoding='cp949') # 1~3 비율 맞춰서 오버샘플링 : 스모트
df2

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,90,50,86,20.0,2,1
1,0,1,120,80,90,27.0,3,1
2,0,1,100,60,112,32.0,2,1
3,0,1,170,120,86,23.0,2,1
4,0,1,135,80,104,35.0,2,1
...,...,...,...,...,...,...,...,...
1481319,0,22,150,90,95,28.0,1,1
1481320,0,14,138,75,178,23.0,1,1
1481321,0,25,118,74,143,21.0,1,1
1481322,1,20,128,79,87,23.0,1,1


In [14]:
df2.drop(df2[df2['Dis']==0].index, inplace=True)
df2

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS,Dis
0,0,1,90,50,86,20.0,2,1
1,0,1,120,80,90,27.0,3,1
2,0,1,100,60,112,32.0,2,1
3,0,1,170,120,86,23.0,2,1
4,0,1,135,80,104,35.0,2,1
...,...,...,...,...,...,...,...,...
1481319,0,22,150,90,95,28.0,1,1
1481320,0,14,138,75,178,23.0,1,1
1481321,0,25,118,74,143,21.0,1,1
1481322,1,20,128,79,87,23.0,1,1


In [15]:
# 1,0 정답 레이블 삭제
df2.drop(['Dis'], axis=1, inplace=True)
df2

,SEX,BTH_G,SBP,DBP,FBS,BMI,DIS
0,0,1,90,50,86,20.0,2
1,0,1,120,80,90,27.0,3
2,0,1,100,60,112,32.0,2
3,0,1,170,120,86,23.0,2
4,0,1,135,80,104,35.0,2
...,...,...,...,...,...,...,...
1481319,0,22,150,90,95,28.0,1
1481320,0,14,138,75,178,23.0,1
1481321,0,25,118,74,143,21.0,1
1481322,1,20,128,79,87,23.0,1


In [16]:
df2.notnull().sum()

SEX      740662
BTH_G    740662
SBP      740662
DBP      740662
FBS      740662
BMI      740662
DIS      740662
dtype: int64

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 740662 entries, 0 to 1481323
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   SEX     740662 non-null  int64  
 1   BTH_G   740662 non-null  int64  
 2   SBP     740662 non-null  int64  
 3   DBP     740662 non-null  int64  
 4   FBS     740662 non-null  int64  
 5   BMI     740662 non-null  float64
 6   DIS     740662 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 45.2 MB


In [19]:
# 데이터 정규화

# 표준 스케일러(평균 0, 분산 1)
scaler = StandardScaler()

# feature와 target 분리(설명변수 및 목표변수 분리)
x = df2.drop(['DIS'], axis=1) # df[df.columns.difference(['target'])]
y = np.array(df2['DIS']) # y = df['target']

# 설명변수 데이터 스케일링
x_train = scaler.fit_transform(x) # x_scaled = scaler.fit_transform(x)

In [20]:
# 전체 데이터에서 학습데이터셋과 테스트셋을 나눔
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, shuffle=True) # 기본 0.25

# train과 test데이터의 비율을 6:4로 나누어주고
# 0.4비율이 된 test데이터를 다시 각 0.5 비율로 validation과 test를 나누어 줌
# 최종 결과가 train : test : validation = 6 : 2 : 2 비율

# # train set에서 train과 validation set를 나눔 # x,y,test_size=0.4
# x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5, shuffle=True)

print(x_train.shape)
print(y_train.shape)
# print(x_val.shape)
# print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(592529, 6)
(592529,)
(148133, 6)
(148133,)


In [21]:
# 랜덤포레스트 without tuning
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
forest_predictions = rf.predict(x_test)

accuracy_score(y_test, forest_predictions)
forest_predictions = rf.predict(x_test)

In [22]:
# 데이터 100만 개 사용
print(confusion_matrix(y_test, forest_predictions))

[[43173  5438  3197]
 [ 6132 37783  4499]
 [ 3299  4248 40364]]


In [23]:
rf = RandomForestClassifier(n_estimators=100, random_state=42).fit(x_train,y_train)

print("train score : {}".format(rf.score(x_train, y_train)))
#print("val score : {}".format(rf.score(x_val, y_val)))

#print("test score : {}".format(dTree.score(x_test,y_test)))
predict_y = rf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(x_test)))
print("test score : {}".format(rf.score(x_test, y_test)))

train score : 0.9707761139117241
              precision    recall  f1-score   support

           1       0.82      0.83      0.83     51808
           2       0.80      0.78      0.79     48414
           3       0.84      0.84      0.84     47911

    accuracy                           0.82    148133
   macro avg       0.82      0.82      0.82    148133
weighted avg       0.82      0.82      0.82    148133

test score : 0.8189937421101308


---

In [24]:
dTree = DecisionTreeClassifier(max_depth=10,random_state=42).fit(x_train,y_train)

print("train score : {}".format(dTree.score(x_train, y_train)))
#print("val score : {}".format(dTree.score(x_val, y_val)))

#print("test score : {}".format(dTree.score(x_test,y_test)))
predict_y = rf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(x_test)))
print("test score : {}".format(rf.score(x_test, y_test)))

train score : 0.589522200601152
              precision    recall  f1-score   support

           1       0.82      0.83      0.83     51808
           2       0.80      0.78      0.79     48414
           3       0.84      0.84      0.84     47911

    accuracy                           0.82    148133
   macro avg       0.82      0.82      0.82    148133
weighted avg       0.82      0.82      0.82    148133

test score : 0.8189937421101308


In [25]:
import graphviz
from sklearn.tree import export_graphviz

export_graphviz(dTree, out_file='tree.dot', class_names= ['1','2','3'], feature_names=df.columns[:-1], impurity=False, filled=True)
with open('tree.dot') as file_reader:
  dot_graph = file_reader.read()

dot = graphviz.Source(dot_graph)
dot.render(filename='tree.png')

'tree.png.pdf'

In [26]:
with open("tree.dot") as f:
  dot_graph = f.read()
graphviz.Source(dot_graph)